# Watermass transformation in MOM6
Mass transport ($G$) across a contour of a materially-conserved tracer ($\lambda$) can be derived from the integrated diffusive tendencies of that tracer ($\dot{\lambda}$). Formally, this is written
$$ G(\lambda) = \frac{\partial}{\partial \lambda}\iiint_{\lambda'\leq\lambda}\rho\dot{\lambda}dV \, .$$
This calculation, initially laid out by Walin (1982) and recently generalized in Groeskamp et al. (2019), is broadly known as watermass transformation and it can be used to reframe the ocean circulation in a new coordinate system.  

Here, we show how to carry out this calculation in MOM6 in a variety of contexts.

## Discrete formulation of watermass transformation for MOM6
For evaluation in an ocean model, we can write out a discrete version of the above equation (see Section 7.5 in Groeskamp et al., 2019):
$$ G(\lambda) = \frac{1}{\Delta\lambda}\sum_n\sum_{i,j,k}\Pi(\lambda_n,\lambda,\Delta\lambda)(\rho\dot{\lambda})_{i,j,k}V_{i,j,k} \, ,$$
where $\Delta\lambda$ is the discrete bin width for defining $\lambda$ contours, $V$ is the grid-cell volume, and we have introduced a boxcar function to accumulate grid cells in which $\lambda$ falls within the discrete bin:
$$
\Pi(\lambda_n,\lambda,\Delta\lambda) = \left\{
                \begin{array}{ll}
                  1 \quad \mathrm{if} \, \lambda_n \in [\lambda-\Delta\lambda/2,\lambda+\Delta\lambda/2] \\
                  0 \quad \mathrm{otherwise} \\
                \end{array}
              \right.
$$
Now, in the case of heat and salt in MOM6, it is the vertically-extensive tracer content that is conserved (see budget closure tutorial) and we can rewrite the discrete equation as:
$$ G(\lambda) = \frac{1}{\Delta\lambda}\sum_n\sum_{i,j,k}\Pi(\lambda_n,\lambda,\Delta\lambda)(\rho\dot{\Lambda})_{i,j,k}A_{i,j} \, ,$$
where $\dot{\Lambda} = \int^{z_{k-1}}_{z_k} \dot{\lambda}\,dz$ is the diffusive tendency of the vertically-integrated tracer content, and $A$ is the horizontal grid area.  

From the equation above, it becomes clear that the calculation simply involves integrating tracer tendencies in discrete bins based on their $\lambda$ value. Here, we will go through several approaches to perform this integration, specifically the accumulation of the terms into $\lambda$ bins.

## Transformation across temperature contours
In the first instance, we consider transformation across contours of constant temperature - a quantity that is materially conserved. At present, online remapping to temperature layers is not possible in MOM6, so the binning procedure has to be performed offline, *i.e.* with the saved diagnostics. Here, we show two different approaches to perform that binning: a simple histogram approach (using the `xhistogram` package), and a higher-order vertical-remapping approach (using the `xgcm` package).

In [ ]:
import xarray as xr
from xhistogram.xarray import histogram

In [ ]:
# Load data

In [ ]:
# Diffusive tendency terms

In [2]:
# Binning and integration

In [ ]:
# Plotting

## Transformation across $\rho_2$ contours
MOM6 has the functionality to output diagnostics on a user-defined $\rho_2$ grid (note that budgets of heat and salt remain closed on this grid, see budget closure tutorial). This is equivalent to the binning procedures performed above being done *online*, thus reducing the possibility for errors due to time averaging.  

However, $\rho_2$ is not a materially-conserved quantity, so transformation evaluated across its contours will always be approximate. In practice, the challenge arises because a diffusive tendency is only well-defined for the locally-referenced potential density ($\rho_l$), which cannot be directly related to the potential density referenced to $2000\,m$ ($\rho_2$). In a separate tutorial, we go through the procedure of calculating transformation across contours of the best approximation of a materially-conserved density variable, neutral density, which involves a correction for its discrepancy to $\rho_l$.  

Here, we evaluate transformation across $\rho_2$ contours using tendencies in $\rho_l$ without attempting to apply any correction, simply to show the procedure applied to an online-binned variable.

In [3]:
# Load data

In [ ]:
# Calculate tendencies of the locally-referenced potential density

In [ ]:
# Binning

## Closing the mass budget within layers
The transformation $G$ represents a mass flux across layers of constant $\lambda$. For a time-varying system within a fixed region ($\mathscr{R}$) with boundaries in the ocean interior ($\partial\mathscr{R}_{int}$) and at the ocean surface ($\delta\mathscr{R}_{surf}$), $G$ will be balanced by temporal mass changes and mass transport across the fixed boundaries (see Section 2.3 in Groeskamp et al., 2019):
$$ G = -\frac{dM}{dt} - \psi + W $$
where
$$M = \iiint_{\mathscr{R}(\lambda'\leq\lambda)} \rho dV\,; \quad \psi = \iint_{\partial\mathscr{R}_{int}(\lambda'\leq\lambda)} (\rho\mathbf{u})\cdot\mathbf{\hat{n}}dR\,; \quad W = \iint_{\partial\mathscr{R}_{surf}(\lambda'\leq\lambda)} \rho F dR\,,$$
are, respectively, the total mass in the region, the mass flux across the interior ocean boundary ($\mathbf{\hat{n}}$ is normal to the boundary), and the mass flux across the ocean surface ($F$ is the local balance of air-sea and ice-sea volume fluxes).